In [4]:
import subprocess
from pydub import AudioSegment


# video_path: 동영상의 경로
# audio_path: 추출한 audio가 저장될 경로
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",  # input
        video_path,
        "-vn",  # video nope: ignore the video
        audio_path,
        "-y",  # 이미 파일이 존재하는 경우 덮어쓰기를 함
    ]
    # subprocess는 python code에서 command line 명령을 사용할 수 있게 해줌
    subprocess.run(command)


track = AudioSegment.from_mp3("./files/podcast.mp3")

# track의 길이 확인: 초 단위로 나옴
# print(track.duration_seconds)

ten_minutes = 10 * 60 * 1000

# pydub의 AudioSegment는 오디오트랙을 리스트처럼 다룰 수 있게 해 줌
# five_minutes = 5 * 60 * 1000
# first_five = track[:five_minutes]
# first_five.export("./files/first_five.mp3", format="mp3")

In [ ]:
import math
import os

# 10분씩 잘랐을 때 자투리를 포함시키기 위해 .ceil 처리
chunks = math.ceil(len(track) / ten_minutes)

# 10분 단위로 자르기 위한 로직
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]

    # chunk가 저장될 path 설정
    chunk_path = f"./files/chunks/chunk_{i}.mp3"
    # chunk_path의 directory 결정
    directory = os.path.dirname(chunk_path)

    # 해당 directory가 없다면 생성
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    # chunk_path에 파일을 저장
    chunk.export(chunk_path, format="mp3")